In [1]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np

c:\Users\Dell\miniconda3\envs\gyenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import gym.spaces as gs

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gs.Box(shape=(4,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        t_0 = (obs[0]** 2 + obs[1]**2)**(0.5)
        t_1 = (obs[2]** 2 + obs[3]**2)**(0.5)
        t_2 = obs[4]/obs[5]
        t_3 = obs[6] or obs[7]
        return [t_0, t_1, t_2, t_3]
env = gym.make("LunarLander-v2")
env = Wrapper(env)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

In [4]:
import pennylane as qml

In [5]:
def ActorLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        "y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, y_weights, z_weights):
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            for wire, y_weight in enumerate(y_weights[layer_idx]):
                qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(0)),
            qml.expval(qml.PauliZ(1)),
            qml.expval(qml.PauliZ(2)),
            qml.expval(qml.PauliZ(3))
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [6]:
def CriticLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        "y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, y_weights, z_weights):
        #print(inputs)
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            for wire, y_weight in enumerate(y_weights[layer_idx]):
                qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3)),
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [7]:
class Actor(nn.Module):
    def __init__(self, n_layers=4):
        super(Actor, self).__init__()
        self.n_qubits = 4
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.action_head = ActorLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.action_head(state)
        outputs = -1 + (outputs - outputs.min()) * (2) / (outputs.max() - outputs.min())
        distribution = Categorical(F.softmax(outputs, dim=-1))
        return distribution


class Critic(nn.Module):
    def __init__(self, n_layers=4):
        super(Critic, self).__init__()
        self.n_qubits = 4
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.critic_head = CriticLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.critic_head(state)
        return outputs

In [8]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns



In [9]:

def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters(), lr = 0.005, amsgrad = True)
    optimizerC = optim.Adam(critic.parameters(), lr = 0.005, amsgrad = True)
    for iter in range(n_iters):
        state, _ = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()
        total_rewards = 0
        done = False

        while not done:
            #env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, term, trunc, _ = env.step(action.cpu().numpy())
            done = term or trunc

            total_rewards += reward
            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, total_rewards))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor.state_dict(), 'actorql.pkl')
    torch.save(critic.state_dict(), 'criticql.pkl')
    env.close()



In [10]:

if __name__ == '__main__':
    if os.path.exists('quantum/actorql.pkl'):
        actor = torch.load('quantum/actorql.pkl')
        print('Actor Model loaded')
    else:
        actor = Actor().to(device)
    if os.path.exists('quantum/criticql.pkl'):
        critic = torch.load('quantum/criticql.pkl')
        print('Critic Model loaded')
    else:
        critic = Critic().to(device)
    trainIters(actor, critic, n_iters=1000)

Iteration: 0, Score: -223.02454063901405
Iteration: 1, Score: -129.93674044244335
Iteration: 2, Score: -96.53573002517247
Iteration: 3, Score: -95.23963010992712
Iteration: 4, Score: -121.66438252337954
Iteration: 5, Score: -293.80352414762433
Iteration: 6, Score: -128.69303124576288
Iteration: 7, Score: -132.33813337169673
Iteration: 8, Score: -235.06601186035087
Iteration: 9, Score: -212.9746583825662
Iteration: 10, Score: -286.9039098297894
Iteration: 11, Score: -143.96266680079125
Iteration: 12, Score: -263.0194843665471
Iteration: 13, Score: -260.19953680466426
Iteration: 14, Score: -261.9647117629022
Iteration: 15, Score: -463.36960156916325
Iteration: 16, Score: -401.15939140946216
Iteration: 17, Score: -413.4528125151914
Iteration: 18, Score: -178.9262889735174
Iteration: 19, Score: -249.3602464219701
Iteration: 20, Score: -118.96032838871051
Iteration: 21, Score: -148.00187125112336
Iteration: 22, Score: -83.91775998858728
Iteration: 23, Score: 10.077055926679279
Iteration: 24